### Importing Necessary libraries

In [1]:
import numpy as np
from scipy.optimize import linprog
import imageio

### Loading the data
Provide path to the 'C.npy', 'y.npy' and 'A_inv.npy' files for all the 3 RGB channels

In [2]:
c_red_path = "C_red.npy" #path to the C.npy file (red)
y_red_path = "y_red.npy" #path to the y.npy file (red)
A_inv_red_path = "A_inv_red.npy" #path to the A_inv.py file (red)

c_green_path = "C_green.npy" #path to the C.npy file (green)
y_green_path = "y_green.npy" #path to the y.npy file (green)
A_inv_green_path = "A_inv_green.npy" #path to the A_inv.py file (green)

c_blue_path = "C_blue.npy" #path to the C.npy file (blue)
y_blue_path = "y_blue.npy" #path to the y.npy file (blue)
A_inv_blue_path = "A_inv_blue.npy" #path to the A_inv.py file (blue)

### The function that solves the given optimization problem
$$
\min _{s}\|s\|_{1} \\
\text{such that } \|y-C s\|_{2}^{2}=0 $$

We solve the given problem using an equivalent linear program. To do so we define:

$$ s = u - v, \; {u}_{i} = \max \left\{ {s}_{i}, 0 \right\}, \; {v}_{i} = \max \left\{ -{s}_{i}, 0 \right\} $$

and in place of $\|y-C s\|_{2}^{2}=0$ we take $y = C s$

Then the problem becomes:

\begin{align*}
\arg \min_{u, v} \: & \: \sum_{i = 1}^{n} {u}_{i} + {v}_{i} \\
\text{subject to} \: & \: C \left( u - v \right) = y \\
& \: u \succeq \boldsymbol{0} \\
& \: v \succeq \boldsymbol{0}
\end{align*}

In [3]:
def solve(A, b):
  numRows = A.shape[0] 
  numCols = A.shape[1]

  print("Number of Rows of A = " + str(numRows))
  print("Number of Columns of A = " + str(numCols))

  vF = np.ones([2*numCols, 1])

  mAeq = np.concatenate((A, -A), axis=1)
  vBeq = b

  vLowerBound = np.full([2 * numCols, 1], 0)
  vUpperBound = np.full([2 * numCols, 1], np.inf)
  Bounds = np.concatenate((vLowerBound, vUpperBound), axis=1)

  result = linprog(vF, A_eq=mAeq, b_eq=vBeq, bounds=Bounds)
  vUV = result.x
  s = vUV[0:numCols] - vUV[numCols:];

  return s

The below code finds the sparse vector 's' using the above function for all 3 RGB channels

In [4]:
A_red = np.load(c_red_path)
b_red = np.load(y_red_path)
s_red = solve(A_red, b_red)
print("Red Done")
print(s_red)
#np.save('s_red', s_red) #if you need to save the sparse vector s, uncomment this line

A_green = np.load(c_green_path)
b_green = np.load(y_green_path)
s_green = solve(A_green, b_green)
print("Green Done")
print(s_green)
#np.save('s_green', s_green) #if you need to save the sparse vector s, uncomment this line

A_blue = np.load(c_blue_path)
b_blue = np.load(y_blue_path)
s_blue = solve(A_blue, b_blue)
print("Blue Done")
print(s_blue)
#np.save('s_blue', s_blue) #if you need to save the sparse vector s, uncomment this line

Number of Rows of A = 3680
Number of Columns of A = 9200
Red Done
[ 1.82918739e+04 -7.58328455e+01 -1.99277691e+03 ... -1.70736424e-07
 -1.27244161e-07  1.41989978e-07]
Number of Rows of A = 3680
Number of Columns of A = 9200
Green Done
[ 1.06283207e+04  9.36923324e+02  2.21130014e+03 ...  1.07811914e-07
 -2.65544772e+01  8.79409114e-08]
Number of Rows of A = 3680
Number of Columns of A = 9200
Blue Done
[ 5.33235341e+03 -1.17917306e+03  5.24387524e+02 ... -3.17361531e-07
 -1.01964851e-07 -3.55131336e+01]


The below code finds the vector 'x' corresponding to the original image using the matrix 'A' for all 3 RGB channels

In [5]:
A_inv_red = np.load(A_inv_red_path)
A_inv_red = A_inv_red.astype('float64')
A_red = np.linalg.inv(A_inv_red) 
x_red = A_red.dot(s_red)
print("Red Done")
print(x_red)
#np.save('x_red', x_red) #if you need to save the vector x, uncomment this line

A_inv_green = np.load(A_inv_green_path)
A_inv_green = A_inv_green.astype('float64')
A_green = np.linalg.inv(A_inv_green) 
x_green = A_green.dot(s_green)
print("Green Done")
print(x_green)
#np.save('x_green', x_green) #if you need to save the vector x, uncomment this line

A_inv_blue = np.load(A_inv_blue_path)
A_inv_blue = A_inv_blue.astype('float64')
A_blue = np.linalg.inv(A_inv_blue) 
x_blue = A_blue.dot(s_blue)
print("Blue Done")
print(x_blue)
#np.save('x_blue', x_blue) #if you need to save the vector x, uncomment this line

Red Done
[138.42724703 179.94747793 211.9727874  ... -11.1121701  -17.01062231
   4.89312927]
Green Done
[171.1169112  236.63477674 243.09956764 ...  -8.81282221   8.64575914
  11.99342035]
Blue Done
[38.50812995 51.69403817 53.20185839 ...  3.7487006  17.73353223
 -4.45735626]


The below code converts the vector 'x' back to image in RGB format using 'imageio' library

*make sure to change the dimensions here properly.* \
*For example if the image has width 92 px and height 100 px, the (100,92) goes in the bracket*

In [6]:
#make sure to change the dimensions here properly. 
#For example if the image has width 92 px and height 100 px, the (100,92) goes in the bracket
y_red = x_red.reshape((100,92), order='F')
y_green = x_green.reshape((100,92), order='F')
y_blue = x_blue.reshape((100,92), order='F')

y = np.zeros((100,92,3))
y[:,:,0] = y_red
y[:,:,1] = y_green
y[:,:,2] = y_blue

imageio.imwrite('imageRGB.jpg', y)

Lossy conversion from float64 to uint8. Range [-215.5875798261548, 455.22381042333035]. Convert image to uint8 prior to saving to suppress this warning.
